# 4.1 FewShotPromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate(
#     template = "What is the capital of {country}?",
#     input_variables=["country"],
# )

# 위방식을 간단하게 아래와 같이 표현할 수 있다.
t = PromptTemplate.from_template("What is the capital of {country}?")

t.format(country="Japan")



'What is the capital of Japan?'

## Fewshot
> `Fewshow`은 모델에게 예제들을 준다는 뜻. 더 나은 대답을 할 수 있도록 하는 예제들을 부여할 수 있다.
결괏값을 말로 풀어 설명하는 것 보다 예제 형식을 제시하는 것이 더 나은 방법이기 때문이다.
예) 고객 지원 시스템에서 고객 문의 응답에 대한 스크립트 기록 같은 것들을 데이터베이스에서 가져와서 Fewshot으로 형식화할 수 있다.

In [4]:
#예제없이 작성 테스트
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:

1. Capital: The capital city of France is Paris, which is also its largest city. Paris is famous for iconic landmarks like the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Champs-Élysées.

2. Language: The official language of France is French, which is spoken by the majority of the population. French is also one of the official languages of many international organizations.

3. History: France has a long and influential history. It was home to the Gauls before being conquered by the Romans. It later became a powerful medieval kingdom and played a significant role in the Renaissance, Enlightenment, and French Revolution.

4. Cuisine: French cuisine is renowned worldwide. It is known for its diverse range of dishes, including pastries, cheeses, wines, and gourmet cuisine. French culinary techniqu

"France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:\n\n1. Capital: The capital city of France is Paris, which is also its largest city. Paris is famous for iconic landmarks like the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Champs-Élysées.\n\n2. Language: The official language of France is French, which is spoken by the majority of the population. French is also one of the official languages of many international organizations.\n\n3. History: France has a long and influential history. It was home to the Gauls before being conquered by the Romans. It later became a powerful medieval kingdom and played a significant role in the Renaissance, Enlightenment, and French Revolution.\n\n4. Cuisine: French cuisine is renowned worldwide. It is known for its diverse range of dishes, including pastries, cheeses, wines, and gourmet cuisine. French culinary

In [22]:
#실제 FewShotPromptTemplate를 사용하여 만들어 보자.
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
# 1. 예제 형식을 지정하기. 형식화 하려면 형식 지정 도구를 만들어야한다.
example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)
# 아래 방식으로 동일하게 작동한다.
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="Human: What do you know about {country}? ",
    input_variables=["country"]
)
# suffix는 형식화 된 모든 예제 마지막에 나오는 내용

prompt.format(country="Germany")

# 이프롬프트를 chain에 넣어보자
chain = prompt | chat

chain.invoke({
    "country": "Japan"
})


I know this:
Capital: Ankara
Language: Turkish
Food: Kebab and Baklava
Currency: Turkish Lira

AIMessageChunk(content='I know this:\nCapital: Ankara\nLanguage: Turkish\nFood: Kebab and Baklava\nCurrency: Turkish Lira')

# 4.2 FewShotChatMessagePromptTemplate
채팅 메시지를 위해 사용..

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# 실제 FewShotPromptTemplate를 사용하여 만들어 보자.
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
# 1. 예제 형식을 지정하기. 형식화 하려면 형식 지정 도구를 만들어야한다.
example_prompt = ChatPromptTemplate.from_messages([
    ("human"," What do you know about {country}?"),
    ("ai", "{answer}")
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

# 이프롬프트를 chain에 넣어보자
chain = final_prompt | chat

chain.invoke({
    "country": "Turkey"
})


I know this:
Capital: Ankara
Language: Turkish
Food: Kebab and Baklava
Currency: Turkish Lira

AIMessageChunk(content='I know this:\nCapital: Ankara\nLanguage: Turkish\nFood: Kebab and Baklava\nCurrency: Turkish Lira')

# 4.3 LengthBasedExampleSelector
다이나믹하게 예제를 선택하는 방법.
많은 예제들이 존재할 수 있는데 예제가 많으면 더많은 prompot비용을 지불해야한다.
이럴 경우 적절한 example들을 골라서 promt에 허용할 것인지 결정해애한다.
그리고 지불할 돈이 있더라도 떄로는 모델에 알맞는 양의 제한이 있기 때문이다.

In [35]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
# 기본적인 예제들을 형식화 할 수 있고 예제의 양이 얼마나 되는지 확인 할 수 있다.
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
import random

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.appenx(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

#실제 FewShotPromptTemplate를 사용하여 만들어 보자.
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
# 1. 예제 형식을 지정하기. 형식화 하려면 형식 지정 도구를 만들어야한다.
# example_template = """
#     Human: {question}
#     AI: {answer}
# """
# example_prompt = PromptTemplate.from_template(example_template)
# 아래 방식으로 동일하게 작동한다.
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=180, #예제의 양을 조절.
# )
example_selector = RandomExampleSelector(
    examples=examples
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples = examples,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}? ",
    input_variables=["country"]
)
# suffix는 형식화 된 모든 예제 마지막에 나오는 내용

prompt.format(country="Brazil")

# 이프롬프트를 chain에 넣어보자
# chain = prompt | chat

# chain.invoke({
#     "country": "Japan"
# })


'Human: What do you know about Italy?\nAI: \n        I Know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil? '

# 4.4 Serialization and Composition
이번장에서는 디스크에서 prompt templates를 가죠오는 방법에 대해서 학습한다.
회사에서 프롬프트 엔지니어링하느 사람이 있고 내가 개발자라면 해당 프롬포트를 가져와야 하는 상황이라던가
어딘가 저장해두고, 다른 누구나 프롬프트를 가져다 쓸수 있도록 하고 싶은 경우에 유용.




### 두가지 타입의 prompt
* Json
* yaml


In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

# chat = ChatOpenAI(
#     temperature=0.1,
#     streaming=True,
#     callbacks=[
#         StreamingStdOutCallbackHandler()
#     ]
# )


'What is the capital of Germany?'

### 많은 prompt들의 memory 등을 다 모으는 방법에 대해 알아볼것이다. 
prompt가 많을 떄 유용

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# PiplinelinePromptTemplate은 많은 prompt들을 하나로 합칠 수 있도록 해준다.
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

full_prompt.format(
    character="Viking",
    example_question="What is your location",
    example_answer="Arrrrg! That is a secret!! Arg arg!!",
    question="What is your fav food? reply in Korean"
)

chain = full_prompt | chat

chain.invoke({
    "character": "Viking",
    "example_question": "What is your location",
    "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
    "question": "What is your fav food?"
})

Me: Ahhh, me favorite food be a hearty feast of roasted boar and freshly baked bread! The taste of victory be sweeter with a full belly, arg arg!

AIMessageChunk(content='Me: Ahhh, me favorite food be a hearty feast of roasted boar and freshly baked bread! The taste of victory be sweeter with a full belly, arg arg!')

# 4.5. Caching